<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_vectors_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_vectors_web_lg')


In [ ]:
!mkdir ./results

In [ ]:
!python /content/drive/MyDrive/KAN/train.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/KAN/train.py", line 2, in <module>
    from core.exec import Execution
  File "/content/drive/.shortcut-targets-by-id/1qq5LCjZ3AGmL2Kv20y4s0nEVp9K7G-Mn/KAN/core/exec.py", line 2, in <module>
    from core.model.net import Net
ModuleNotFoundError: No module named 'core.model.net'


In [ ]:
!mkdir ./data
!cp /content/drive/MyDrive/data_path.zip ./data/
!cd ./data && unzip data_path.zip

In [ ]:
!cd ./data && unzip data_path.zip

In [ ]:
# !ls /content/data/data/ques_embeddings/bluebert
!cp -r /content/data/data/ques_embeddings/w2v /content/drive/MyDrive/VQA_path/ques_embeddings

In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy

/content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy


In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo | wc -l

32761


In [ ]:
!ls /content/data/data/ques_embeddings/bioelmo/7563.npy

/content/data/data/ques_embeddings/bioelmo/7563.npy


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

In [ ]:
class Image_Question_Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding,
                 rate=0.1 ,pretrained_cnn_type='inception', multiplier = 10):
        super(Image_Question_Encoder, self).__init__()
        self.maximum_position_encoding = maximum_position_encoding
        self.pretrained_CNN = pretrained_cnn(pretrained_cnn_type)
        self.fc1 = Dense(maximum_position_encoding * multiplier) # fully connected layer
        self.image_dense = Dense(d_model, activation='relu')
        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=maximum_position_encoding)
        self.concatenator = Concatenate(axis=1)
        self.conv = tf.keras.layers.Conv2D(1024, 3, padding='same', activation='relu')
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True))

    def call(self, question, img, training, mask):
        img_feature = self.pretrained_CNN(img)  # (7, 7, 1024)
        img_feature = self.conv(img_feature)
        img_shape = img_feature.shape[-1]
        img_feature = tf.keras.layers.Reshape((-1, img_shape))(img_feature)  # (49, 1024)
        img_feature = self.fc1(img_feature)  # (49, pe_input)
        img_feature = tf.transpose(img_feature, perm=(0, 2, 1))  # (Res, 49)
        img_feature = tf.keras.layers.Reshape((self.maximum_position_encoding, -1))(img_feature)
        img_feature = self.image_dense(img_feature)  # (pe_input, 512)
        question = self.bilstm(question)
        context = self.question_encoder(question, img_feature, training, mask)  # (batch, question_len, 512)
        # context = self.concatenator([img_feature, question_feature]) # （batch, 64 + question_len, 512)
        return context  # (64, 108, 512)

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

# Knowledge

## SenticNet

In [2]:
pip install senticnet

     |████████████████████████████████| 51.9MB 142kB/s 


In [3]:
from senticnet.senticnet import SenticNet

sn = SenticNet()
concept_info = sn.concept('love')
polarity_label = sn.polarity_label('love')
polarity_value = sn.polarity_value('love')
moodtags = sn.moodtags('love')
semantics = sn.semantics('love')
sentics = sn.sentics('love')

In [6]:
print(concept_info)
print(polarity_label)
print(polarity_value)
print(moodtags)
print(semantics)
print(sentics)

{'polarity_label': 'positive', 'polarity_value': '0.83', 'sentics': {'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}, 'moodtags': ['#joy', '#eagerness'], 'semantics': ['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']}
positive
0.83
['#joy', '#eagerness']
['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']
{'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}


## ConceptNet

In [7]:
pip install conceptnet-lite

     |████████████████████████████████| 51kB 2.4MB/s 
     |████████████████████████████████| 2.3MB 6.5MB/s 
     |████████████████████████████████| 870kB 27.5MB/s 
  Created wheel for peewee: filename=peewee-3.14.4-cp37-cp37m-linux_x86_64.whl size=593002 sha256=e54a1d57de6c9ecefd59a57393d1ed2f62c47115211e09bab29d8c971426f907
  Stored in directory: /root/.cache/pip/wheels/a0/ce/ff/732bd2a2729233b3a55bb5d8557d500f7be41bd21df3f01048
  Created wheel for lmdb: filename=lmdb-0.97-cp37-cp37m-linux_x86_64.whl size=219753 sha256=44bc074d59a3cae9c2be5a8b720a5fd989d5d4d76583001f00b3c6aed7da8d3b
  Stored in directory: /root/.cache/pip/wheels/7f/4d/44/b299a9b22d8eaaaa455304687fb6d4c02d9178c1530850e12a
Successfully built peewee lmdb
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99


In [8]:
import conceptnet_lite

conceptnet_lite.connect("/path/to/conceptnet.db")

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


File not found: /path/to/conceptnet.db
Download compressed database
[*] 0 bytes / 1.85 GB @ 0 bytes/s [------------------] [0.0%, 0s left]    [*] 1.8 MB / 1.85 GB @ 9.1 MB/s [------------------] [0.1%, 0s left]    [*] 11.0 MB / 1.85 GB @ 36.8 MB/s [------------------] [0.6%, 0s left]    [*] 22.2 MB / 1.85 GB @ 55.5 MB/s [------------------] [1.2%, 0s left]    [*] 31.5 MB / 1.85 GB @ 63.1 MB/s [------------------] [1.7%, 0s left]    [*] 41.1 MB / 1.85 GB @ 68.5 MB/s [------------------] [2.2%, 0s left]    [*] 50.2 MB / 1.85 GB @ 71.7 MB/s [-------------

In [9]:
from conceptnet_lite import Label

cat_concepts = Label.get(text='cat', language='en').concepts  
for c in cat_concepts:
    print("    Concept URI:", c.uri)
    print("    Concept text:", c.text)
    print("    Concept language:", c.language.name)

    Concept URI: /c/en/cat
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wn/animal
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wn/person
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wp/novel
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/v/wn/contact
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/a/wikt/en_4
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wikt/en_1
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wikt/en_2
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wikt/en_3
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wikt/en_5
    Concept text: cat
    Concept language: en
    Concept URI: /c/en/cat/n/wikt/en_6
    Concept text: cat
    Concept lan

In [18]:
from conceptnet_lite import Label, edges_for

d = {}
for e in edges_for(Label.get(text='brain', language='en').concepts, same_language=True):
    # print(e.start.text, "::", e.end.text, "|", e.relation.name, "|", e.etc['weight'])
    d[(e.start.text,e.relation.name,e.end.text)] = e.etc['weight']
    # print(e.etc['weight'])

In [21]:
for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
  if i == 5:
    break
  print(f)

(('thinking', 'has_prerequisite', 'brain'), 6.0)
(('mind', 'related_to', 'brain'), 5.142)
(('remembering', 'has_prerequisite', 'brain'), 4.899)
(('brain', 'part_of', 'body'), 4.472)
(('brain', 'used_for', 'thinking'), 4.472)
